In [2]:
import zipfile
import numpy as np
import tensorflow as tf

path_to_glove = "path/to/glove/file"
PRE_TRAINED = True
GLOVE_SIZE = 300
batch_size = 128
embedding_dimension = 64
num_classes = 2
hidden_layer_size = 32
times_steps = 6

In [8]:
Sample_cnt = 10000
odd_sentences=[]
even_sentences=[]
seqlens=[]

digit_to_word_map = {1:"One",2:"Two",3:"Three",4:"Four",5:"Five",6:"Six",7:"Seven",8:"Eight",9:"Nine",}
digit_to_word_map[0]="PAD"
for i in range(Sample_cnt):
    rand_seq_len = np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2), rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2), rand_seq_len)

    # 패딩
    if rand_seq_len < 6:
        rand_odd_ints = np.append(rand_odd_ints, [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints, [0]*(6-rand_seq_len))
    odd_sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    even_sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))
data = odd_sentences + even_sentences
len(odd_sentences),len(even_sentences),len(data)

# 홀수, 짝수 시퀀스의 seq 길이(2배 증폭)
seqlens*=2

# 단어를 인덱스에 매핑
word2index_map = {}
index = 0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1

# 역방향 매핑
index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)
print('word2index_map :{}'.format(word2index_map))
print('index2word_map :{}'.format(index2word_map))
print('vocabulary_size :{}'.format(vocabulary_size))

# one hot encoding : 짝수=[1,0], 홀수=[0,1]
labels = [1]*Sample_cnt + [0]*Sample_cnt
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding
print('앞쪽')
print('   >>> data : {}'.format(data[:3]))
print('   >>> one-hot-encoding : {}'.format(labels[:3]))
print('뒤쪽')
print('   >>> data : {}'.format(data[Sample_cnt*2-4:-1]))
print('   >>> one-hot-encoding : {}'.format(labels[Sample_cnt*2-4:-1]))

word2index_map :{'one': 0, 'three': 1, 'five': 2, 'nine': 3, 'seven': 4, 'pad': 5, 'eight': 6, 'four': 7, 'six': 8, 'two': 9}
index2word_map :{0: 'one', 1: 'three', 2: 'five', 3: 'nine', 4: 'seven', 5: 'pad', 6: 'eight', 7: 'four', 8: 'six', 9: 'two'}
vocabulary_size :10
앞쪽
   >>> data : ['One One Three One Three Five', 'Five Nine Three Three Seven One', 'Five One Five PAD PAD PAD']
   >>> one-hot-encoding : [[0, 1], [0, 1], [0, 1]]
뒤쪽
   >>> data : ['Eight Six Eight PAD PAD PAD', 'Four Two Four Eight Eight Four', 'Six Two Six Two PAD PAD']
   >>> one-hot-encoding : [[1, 0], [1, 0], [1, 0]]


In [10]:
def get_glove(path_to_glove, word2index_map):
    embedding_weights = {}
    count_all_words = 0
    with zipfile.ZipFile(path_to_glove) as z:
        with z.open("glove.840B.300d.txt") as f:
            for line in f:
                vals = line.split()
                word = str(vals[0].decode("utf-8"))
                if word in word2index_map:
                    print(word)
                    count_all_words += 1
                    coefs = np.asarray(vals[1:], dtype='float32')
                    coefs /= np.linalg.norm(coefs)
                    embedding_weights[word] = coefs
                if count_all_words == len(word2index_map) - 1:
                    break
    return embedding_weights
word2embedding_dict = get_glove(path_to_glove, word2index_map)


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/glove/file'